In [1]:
import os
import pandas as pd
from Bio import SeqIO
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Seq import Seq
import csv
import pandas as pd
from pdf2image import convert_from_path
from PIL import ImageOps, Image
from ete3 import Tree, TreeNode, TreeStyle, NodeStyle, faces, RectFace

## Generate lists of target locus ID's 
This pulls from the strictly defined pyl user list (i.e. a genome with complete pylRS, pylT, and pylBCD), and identifies locus ID's for  HMM hits of interest. For the biosynthetic machinery, we're using pylB as a representative since the majority of the pyl biosynethic machinery occurs in an operon. For those genomes where pylB is located far away from pylC and pylD, we manually search and generate the neighborhood.

In [2]:
pwd

'/Users/katharineshalvarjian/Documents/gtdb/3-notebooks'

In [3]:
# create score dictionary for each HMM
score_dict = {}

for file in os.listdir("../1-pyl/pyl/pyl-hmms/"):
    if ".DS_Store" in file:
        continue
    with open(f"../1-pyl/pyl/pyl-hmms/{file}", 'r') as f: 
        for line in f: 
            if line.startswith('TC'): 
                gene = file.split('.')[0]
                score = float(line.strip().split( )[1])
                score_dict.update({gene:score})

In [4]:
df = pd.read_csv("../analysis_files/2025-02-11_pyl_STRICT.csv")
genomes = df['genome'].tolist()

# make loci lists 
pylB_loci = []
pylS_loci = []
pylSn_loci = []
pylSc_loci = []

genes = ['pylB', 'pylS', 'pylSn', 'pylSc']

for genome in genomes:
    genome_df = pd.read_csv(f"../1-pyl/pyl/pyl-hmm_hits/{genome}_hmm_hits.csv", sep=",")
    
    for gene in genes:
        score = score_dict[gene]
        filtered_df = genome_df[(genome_df['hmm'] == gene) & (genome_df['score'] >= score)]
        
        if not filtered_df.empty:
            gene_id = filtered_df.iloc[0]['geneID']
            
            if gene == 'pylB':
                pylB_loci.append(gene_id)
            elif gene == 'pylS':
                pylS_loci.append(gene_id)
            elif gene == 'pylSc':
                pylSc_loci.append(gene_id)
            elif gene == 'pylSn':
                pylSn_loci.append(gene_id)


## Make an empty diagram
Generates an empty/blank .pdf genome diagram to use for nodes that lack components

In [62]:
gd_diagram = GenomeDiagram.Diagram()
gd_track_for_features = gd_diagram.new_track(1, tracklines=0)
gd_feature_set = gd_track_for_features.new_set()

gd_diagram.draw(format="linear", orientation="landscape", fragments=1, start=0, end=1, pagesize=(30*cm, 7.5*cm))
gd_diagram.write("./dummy.pdf", "PDF")

## Generate neighborhood images 
Generates genomic neighborhood images (in .pdf format) for each target gene and all genes within a genomic window around that target gene. Note that you should be able to export these with a .png extension, but BioPython was having difficulty loading a PIL sub-package, so I used a pdf > png converter work-around. 
This happens in several steps: 
1. define a get_features functions, which returns all features within the specified window length. This takes in the record (a contig), the target feature ID, and a window length and returns all features within that window. Note that because genbank files re-index contigs at 1, isolating the record is essential (otherwise it will an excess of features within the window).
2. define a pad_sequence function, which pads sequences with N's when the window length exceeds the contig length. This takes a record (contig), target start, and target end.
3. run these functions within a main loop to get features, pad sequences (if neccessary) and generate the output pdf's. Note the 'to_flip' list -- this generates a list of image ID's that we will flip the orientation of (mirror), such that mmp16 is on the forward strand. Because many of these features are on contigs, the assigned orientation is not meaningful.
4. convert the pdf's to png's and mirror those images that are listed in 'to_flip'.

In [5]:
def get_features(record, target_feature, window_length):
    target_start = max(0, target_feature.location.start - window_length)
    target_end = min(len(record.seq), target_feature.location.end + window_length)
    features = []

    for feature in record.features: 
        if feature.type == "CDS" and feature != target_feature: 
            if target_start <= feature.location.end and feature.location.start <= target_end: 
                features.append(feature)
        
    return features

In [6]:
def pad_sequence(record, target_start, target_end):
    sequence_length = len(record.seq)
    if target_start < 0:
        left_padding = abs(target_start)
    else:
        left_padding = 0

    if target_end > sequence_length:
        right_padding = target_end - sequence_length
    else:
        right_padding = 0

    padded_sequence = "N" * left_padding + str(record.seq) + "N" * right_padding
    return padded_sequence

In [10]:
cd ../1-pyl/neighborhood/

/Users/katharineshalvarjian/Documents/gtdb/1-pyl/neighborhood


In [12]:
# sets EC number for genes of interest
pylBCD = ['5.4.99.58', '6.3.2.-', '1.4.1.-']
pylS = ['6.1.1.26']
mtase = ['2.1.1.247', '2.1.1.248', '2.1.1.249', '2.1.1.250']
hyd = ['3.5.3.9']

# sets colors for all genes of interest
color_map = {
    'pylBCD': colors.Color(.118, .533, .898, .8),  # blue
    'pylS': colors.Color(1.0, .655, .149, .8),  # orange
    'mtase': colors.Color(.804, .863, .224, .8),  # green
    'hyd': colors.Color(0.0, .537, .482, .8), # teal
    'default': colors.Color(.85, .85, .85, .8) # light gray
}

genome_directory = '../../archaea_prokka/'

# loop through each genome in the list defined as having target gene
for genome in genomes:  
    # create genome diagram
    gd_diagram = GenomeDiagram.Diagram(name=None, tracklines=False)
    path = f"{genome_directory}{genome}/{genome}.gbk"

    # parse genbank records for each genome
    records = list(SeqIO.parse(path, "genbank"))
    target_feature = None
    target_record = None

    # loop through records and compare to the list of target loci, once found set the contig and feature as found  
    for record in records:
        for feature in record.features:
            if feature.type == "CDS" and feature.qualifiers.get('locus_tag', ['N/A'])[0] in pylSc_loci:
                target_record = record
                target_feature = feature
                break

    # once target feature is found, set a window length beginning at the start and stop of the target gene          
    if target_feature:
        window_length = 7500 
        target_start = target_feature.location.start - window_length
        target_end = target_feature.location.end + window_length

        # if window exceeds the contig length, create a padded sequence record & get features
        if target_start < 0 or target_end > len(record.seq):
            padded_sequence = pad_sequence(target_record, target_start, target_end)
            padded_record = SeqIO.SeqRecord(
                Seq(padded_sequence),
                id=target_record.id,
                name=target_record.name,
                description=target_record.description
            )
            features = get_features(target_record, target_feature, window_length)
            padded_record.features = features
        # otherwise, get features
        else:
            features = get_features(target_record, target_feature, window_length)

        # once features are found, create and fill a new track
        if features:
            gd_track = gd_diagram.new_track(1, name="Features", start=target_start, end=target_end, hide=False, greytrack=False, scale=False)
            gd_feature_set = gd_track.new_set()

            pylBCD_features = []
            pylS_features = []
            mtase_features = []
            hyd_features = []
            other_features = []

            # define target features for track 
            for feature in features:
                if feature != target_feature:
                    locus_tag = feature.qualifiers.get('locus_tag', ['N/A'])[0]
                    ec_number = feature.qualifiers.get('EC_number', ['N/A'])[0]
                    gene = feature.qualifiers.get('gene', ['N/A'])[0]
                    product = feature.qualifiers.get('product', ['N/A'])[0]

                    if ec_number in pylBCD:
                        pylBCD_features.append((feature, gene))
                    elif ec_number in pylS:
                        pylS_features.append((feature, gene))
                    elif ec_number in mtase:
                        mtase_features.append((feature, gene))
                    elif ec_number in hyd: 
                        hyd_features.append((feature, ec_number))
                    else:
                        other_features.append((feature, product))

            for feature, gene in pylBCD_features:
                gd_feature_set.add_feature(feature, sigil="ARROW",
                                           color=color_map['pylBCD'],
                                           label=False, label_size=8, name=gene, arrowshaft_height=1.0)

            for feature, gene in pylS_features:
                gd_feature_set.add_feature(feature, sigil="ARROW",
                                           color=color_map['pylS'],
                                           label=False, label_size=8, name=gene, arrowshaft_height=1.0)

            for feature, gene in mtase_features:
                gd_feature_set.add_feature(feature, sigil="ARROW",
                                           color=color_map['mtase'],
                                           label=False, label_size=8, name=gene, arrowshaft_height=1.0)

            for feature, gene in hyd_features:
                gd_feature_set.add_feature(feature, sigil="ARROW",
                                           color=color_map['hyd'],
                                           label=False, label_size=8, name=gene, arrowshaft_height=1.0)

            for feature, ec_number in other_features:
                gd_feature_set.add_feature(feature, sigil="ARROW",
                                           color=color_map['default'],
                                           label=False, label_size=8, name=product, arrowshaft_height=1.0)

            gd_feature_set.add_feature(target_feature, sigil="ARROW",
                                       color=color_map['pylBCD'],
                                       label=False, label_size=8, name='pylB', arrowshaft_height=1.0)

            # set page size, draw and write diagram
            pagesize = (30 * cm, 7.5 * cm)
            gd_diagram.draw(format="linear", orientation="landscape",
                            fragments=1, start=target_start, end=target_end, pagesize=pagesize, tracklines=False)
            gd_diagram.write(f"./graphics-pylSc_test/{genome}.pdf", "pdf")

            # if the target feature needs to be flipped, flag for flipping with converted to .png
            # if target_feature.location.strand == -1: 
            #     to_flip.append(f"{genome}.pdf")


In [22]:
import os
from pdf2image import convert_from_path

def convert_pdf_to_png(pdf_folder, png_folder):
    os.makedirs(png_folder, exist_ok=True)

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            images = convert_from_path(pdf_path, dpi=300)

            for i, image in enumerate(images):
                # Save the image as a PNG
                png_file = pdf_file.replace(".pdf", ".png")  # Adding index for multi-page PDFs
                png_path = os.path.join(png_folder, png_file)
                image.save(png_path, "PNG", quality=95, optimize=True)

convert_pdf_to_png("./graphics-pylB_nolabel/", "./png-pylB_nolabel/")
# convert_pdf_to_png("./graphics-pylS/", "./png-pylS/")
# convert_pdf_to_png("./graphics-15000-2/", "./png-15000-2/")

## Make the tree
The functions and loops below generate the tree with gene neighborhood images and presence/absence matrix information overlaid. First, we define a layout function that is applied to nodes using ete3, then we generate the tree itself, loading in information as dataframes and dictionaries. Please note that to use this function, you'll need to define a tax_dict that converts between the GTDB accession (sans first 4 characters, i.e. in 'GCF_' or 'GCA_' format) and the species name. You will also have to define a presence_absence_dict. 

In [1]:
def layout(node):
    # checks if node is a leaf
    if node.is_leaf():
        try:
            # pull node name, convert to species name, add a text_face, & remove the original node ID
            node_name = str(node.name[3:])

            if node.name in tax_dict:
                label = tax_dict[node.name]
                node_label = label.split(";s__")[1]
                text_face = faces.TextFace(f"{node_label}   ({node_name})", fsize=125)
                text_face.margin_left = 5
                text_face.margin_bottom = 10
                faces.add_face_to_node(text_face, node, column=0, position="aligned")


            img_path = f"./png-pylB_nolabel/{node_name}.png"
            img_face = faces.ImgFace(img_path, width=1600, height=300)
            img_face.margin_left = 30
            faces.add_face_to_node(img_face, node, column=1, position="aligned")


    # format line widths and colors for each node
    nstyle = NodeStyle()
    nstyle["fgcolor"] = "#ffffff"
    nstyle["size"] = 0
    nstyle["vt_line_color"] = "#000000"
    nstyle["vt_line_width"] = 10
    nstyle["hz_line_color"] = "#000000"
    nstyle["hz_line_width"] = 10

    node.set_style(nstyle)

# make a taxonomy converter
df2 = pd.read_csv("../../archaea_info/ar53_metadata_tax.csv")
df2["species"] = df2["gtdb_taxonomy"].str.split(";s__", expand=True)[1]
tax_dict = dict(zip(df2["gtdb_accession"], df2["gtdb_taxonomy"]))

# load tree
tree = Tree("./2025-02-05_pylBCD_checkM95-strict.newick", format=1)

# apply tree style to the parsed tree
ts = TreeStyle()
ts.layout_fn = layout
ts.scale = 5000.0
ts.force_topology = False
ts.draw_guiding_lines = True
ts.margin_left = 1000
ts.margin_right = 1000
ts.margin_top = 1000
ts.margin_bottom = 1000

# render tree
tree.render("./neighborhood-7500.pdf", tree_style=ts)
